In [ ]:
# For Numerical Python
import numpy as np
# For Panel Data Analysis
import pandas as pd
#if you want to create a profile report 
from pandas_profiling import ProfileReport
# For Data Visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# To Disable Warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_colwidth', None)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.precision', 4)

In [3]:
import re
import nltk

In [4]:
from wordcloud import WordCloud

In [5]:
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [7]:
data = pd.read_csv("C:/Users/suddutta/Downloads/zomato.csv")

In [8]:
data.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          51717 non-null  object
 1   address                      51717 non-null  object
 2   name                         51717 non-null  object
 3   online_order                 51717 non-null  object
 4   book_table                   51717 non-null  object
 5   rate                         43942 non-null  object
 6   votes                        51717 non-null  int64 
 7   phone                        50509 non-null  object
 8   location                     51696 non-null  object
 9   rest_type                    51490 non-null  object
 10  dish_liked                   23639 non-null  object
 11  cuisines                     51672 non-null  object
 12  approx_cost(for two people)  51371 non-null  object
 13  reviews_list                 51

In [42]:
#rate can't be object. it should be float
#cost should also be integer

In [12]:
data.describe(include = 'all')

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
count,51717,51717,51717,51717,51717,43942,51717.0000,50509,51696,51490,23639,51672,51371,51717,51717,51717,51717
unique,51717,11495,8792,2,2,64,NaN,14926,93,93,5271,2723,70,22513,9098,7,30
top,https://www.zomato.com/bangalore/halli-oota-st...,Delivery Only,Cafe Coffee Day,Yes,No,NEW,NaN,080 43334321,BTM,Quick Bites,Biryani,North Indian,300,[],[],Delivery,BTM
freq,1,128,96,30444,45268,2208,NaN,216,5124,19132,182,2913,7576,7595,39617,25942,3279
mean,NaN,NaN,NaN,NaN,NaN,NaN,283.6975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,803.8389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,7.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,41.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,198.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#cafe coffee day most number of times
# rate has missing values
#phone number has missing values
#missing values in location, rest_type, dish liked, cuisines, cost

In [35]:
null_dict = {}
null_percent = {}
for i in data.columns.values:
    null_dict[i] = data[i].isnull().sum()
    null_percent[i] = round(data[i].isnull().sum()/len(data),2)*100

In [36]:
null_df = (pd.DataFrame({'Missing Values':null_dict, 'Missing Percentage':null_percent})).transpose()
null_df

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
Missing Values,0.0,0.0,0.0,0.0,0.0,7775.0,0.0,1208.0,21.0,227.0,28078.0,45.0,346.0,0.0,0.0,0.0,0.0
Missing Percentage,0.0,0.0,0.0,0.0,0.0,15.0,0.0,2.0,0.0,0.0,54.0,0.0,1.0,0.0,0.0,0.0,0.0


In [37]:
#for sentiment analysis we need the reviews and ratings. rest columns are not important for the sentiment part though 
#few columns are necessary for data exploration

In [38]:
#url, address, phone number, location are redundant and hence we will drop them at the beginnig itself

In [39]:
data.drop(['url', 'address', 'phone', 'location'], axis = 1, inplace = True)

In [40]:
data.shape

(51717, 13)

In [43]:
#correcting rate and cost columns by changing their datatype
#first let's find out the values they posess to understand the correction we need to do

In [60]:
pd.DataFrame(data['rate'].value_counts()).transpose()
#so basically it has ratings given out of 5. we can simply remove '/5' part to get the actual ratings since all are
#out of 5

,NEW,3.9/5,3.8/5,3.7/5,3.9 /5,3.8 /5,3.7 /5,3.6/5,4.0/5,4.0 /5,3.6 /5,4.1/5,4.1 /5,3.5/5,3.5 /5,3.4/5,3.4 /5,3.3/5,4.2 /5,3.3 /5,4.2/5,3.2/5,4.3 /5,3.2 /5,3.1/5,4.3/5,3.1 /5,4.4 /5,3.0/5,4.4/5,3.0 /5,2.9/5,4.5 /5,2.9 /5,2.8/5,2.8 /5,4.5/5,4.6 /5,2.7/5,2.6/5,2.7 /5,4.6/5,2.6 /5,4.7 /5,4.7/5,-,2.5 /5,2.5/5,4.8 /5,2.4/5,4.9 /5,2.4 /5,2.3/5,4.9/5,4.8/5,2.3 /5,2.2/5,2.1 /5,2.1/5,2.0 /5,2.2 /5,2.0/5,1.8 /5,1.8/5
rate,2208,2098,2022,2011,1874,1851,1810,1773,1609,1574,1543,1474,1474,1431,1353,1259,1217,1168,1165,1142,1019,1006,917,867,862,776,699,628,558,519,465,427,409,375,313,287,247,175,170,143,137,125,117,86,81,69,56,45,43,40,30,30,28,25,23,23,19,13,11,7,7,4,3,2


In [ ]:
#so rate column has ratings as float as well as NEW which means those restaurants have no ratings so far as they
#have opened recently
#also there are 15% missing values in the rate column

In [73]:
data[data.rate == 'NEW']['reviews_list'] #so basically 721 new restaurants with few restaurants with multiple
#outlets

72       [('Rated 4.0', 'RATED\n  cost for chicken roll...
75                                                      []
110      [('Rated 1.0', 'RATED\n  I have been here many...
130                                                     []
131                                                     []
                               ...                        
51585                                                   []
51586    [('Rated 5.0', 'RATED\n  This is an awesome pl...
51603    [('Rated 3.0', 'RATED\n  A highly expensive pl...
51604    [('Rated 3.0', 'RATED\n  Biriyani had ordered ...
51605                                                   []
Name: reviews_list, Length: 2208, dtype: object

In [70]:
data[data.rate == '-']['name'].value_counts() #69 restaurants with no ratings

Pailwan House                        1
Mezban Family Restaurant             1
Sugary Bakers - A Boutique Bakery    1
ThugLife Rolls                       1
Mama Mys Cafe                        1
                                    ..
Maruti Garden                        1
Guntur's Vandanam Food Zone          1
Aahar Cafe                           1
Super Chef's (New Royal treat)       1
Village Kitchen                      1
Name: name, Length: 69, dtype: int64

In [ ]:
#this rate column is rating given by zomato

In [74]:
#in reviews_list column we have rating given by customers alongside reviews. so for sentiment analysis we can go with 
#that column

Now we have many restaurants and doing sentiment analysis for all makes no sense. we can may be find out the top rated


In [98]:
data['name'].nunique()#8792 unique restaurants with many outlets 

8792

In [99]:
#to do sentiment analysis we need to focus on any single outlet to understand how it is serving its customers

In [108]:
data.duplicated().any()

True

In [115]:
#let's say we do sentiment analysis for one brand which has most number of outlets implying it's very famous

In [589]:
data.rename(columns= {'listed_in(type)': 'Type', 'listed_in(city)': 'City', 'reviews_list' : 'Reviews'}, inplace = True)

In [569]:
CCD = data[data.name == 'Cafe Coffee Day']                   #96 outlets
CCD.shape

(96, 13)

In [570]:
CCD.head()

,name,online_order,book_table,rate,votes,rest_type,dish_liked,cuisines,approx_cost(for two people),Reviews,menu_item,Type,City
22,Cafe Coffee Day,No,No,3.6/5,28,Cafe,NaN,"Cafe, Fast Food",900,"[('Rated 4.0', 'RATED\n This outlet is better...",[],Cafes,Banashankari
254,Cafe Coffee Day,Yes,No,3.2/5,17,Cafe,NaN,"Cafe, Fast Food",900,"[('Rated 5.0', 'RATED\n Delicious mouthwateri...",[],Delivery,Banashankari
625,Cafe Coffee Day,No,No,NaN,0,Cafe,NaN,"Cafe, Fast Food",900,"[('Rated 4.0', 'RATED\n Had ordered devil?s o...",[],Dine-out,Banashankari
889,Cafe Coffee Day,No,No,3.6/5,19,Cafe,NaN,"Cafe, Fast Food",900,"[('Rated 4.0', ""RATED\n It is Okay place. Thi...",[],Cafes,Bannerghatta Road
1186,Cafe Coffee Day,Yes,No,3.7/5,26,Cafe,NaN,"Cafe, Fast Food",900,"[('Rated 3.0', 'RATED\n What can someone writ...","[""Roasted Chicken 'n' Cheese Multigrain"", 'Dut...",Delivery,Bannerghatta Road


In [571]:
#renaming few column

#CCD.rename(columns= {'listed_in(type)': 'Type', 'listed_in(city)': 'City', 'reviews_list' : 'Reviews'}, inplace = True)

In [572]:
CCD.columns.values

array(['name', 'online_order', 'book_table', 'rate', 'votes', 'rest_type',
       'dish_liked', 'cuisines', 'approx_cost(for two people)', 'Reviews',
       'menu_item', 'Type', 'City'], dtype=object)

In [137]:
#for each CCD outlet we can have a sentiment model which can be used for future prediction

In [138]:
#let's say we choose the first oulet

In [532]:
CCD1 = CCD.iloc[0,]
#CCD1

In [533]:
CCD1.Reviews

'[(\'Rated 4.0\', \'RATED\\n  This outlet is better than other outlets. But the service was slow. Ambience was good. The garlic bread was not that great. But the other dishes were yum.\'), (\'Rated 1.0\', "RATED\\n  Quality of coffee has gone worse. So so so diluted coffee.. Though I ordered cold coffee, it was not at all cold. Even AC seemed to be off in this summer as we felt really hot through out our time. Instead of paying service taxes n getting horrible coffee n sweating, it\'s better to buy coke or drink water or visit better cafes."), (\'Rated 3.0\', "RATED\\n  It\'s a nice place to hangout because ambience is very relaxing. They serve different varieties of coffee. And recently they\'ve added sandwiches burgers . But they\'re not UpTo the mark. The only thing I like here is toastizza!! Not worth the price."), (\'Rated 3.5\', \'RATED\\n  Good place to hangout with buddies, friendly staff but service could be improved. I ordered Cool Blue beverage, sizzle dazzle brownie and bur

In [534]:
CCD1.Reviews= CCD1.Reviews.replace("RATED\\n", "")

In [535]:
CCD1.Reviews = CCD1.Reviews.replace("\'", "")

In [536]:
type(CCD1.Reviews)

str

In [537]:
CCD1.Reviews

'[(Rated 4.0,   This outlet is better than other outlets. But the service was slow. Ambience was good. The garlic bread was not that great. But the other dishes were yum.), (Rated 1.0, "  Quality of coffee has gone worse. So so so diluted coffee.. Though I ordered cold coffee, it was not at all cold. Even AC seemed to be off in this summer as we felt really hot through out our time. Instead of paying service taxes n getting horrible coffee n sweating, its better to buy coke or drink water or visit better cafes."), (Rated 3.0, "  Its a nice place to hangout because ambience is very relaxing. They serve different varieties of coffee. And recently theyve added sandwiches burgers . But theyre not UpTo the mark. The only thing I like here is toastizza!! Not worth the price."), (Rated 3.5,   Good place to hangout with buddies, friendly staff but service could be improved. I ordered Cool Blue beverage, sizzle dazzle brownie and burger were so yummy. I would definitely recommend this place.\\n

In [538]:
Reviews = CCD1.Reviews.strip('[]')#convert from string of list to str

In [539]:
Reviews

'(Rated 4.0,   This outlet is better than other outlets. But the service was slow. Ambience was good. The garlic bread was not that great. But the other dishes were yum.), (Rated 1.0, "  Quality of coffee has gone worse. So so so diluted coffee.. Though I ordered cold coffee, it was not at all cold. Even AC seemed to be off in this summer as we felt really hot through out our time. Instead of paying service taxes n getting horrible coffee n sweating, its better to buy coke or drink water or visit better cafes."), (Rated 3.0, "  Its a nice place to hangout because ambience is very relaxing. They serve different varieties of coffee. And recently theyve added sandwiches burgers . But theyre not UpTo the mark. The only thing I like here is toastizza!! Not worth the price."), (Rated 3.5,   Good place to hangout with buddies, friendly staff but service could be improved. I ordered Cool Blue beverage, sizzle dazzle brownie and burger were so yummy. I would definitely recommend this place.\\n\

In [540]:
type(Reviews)

str

In [541]:
Reviews = Reviews.split("), (")

In [542]:
Reviews

['(Rated 4.0,   This outlet is better than other outlets. But the service was slow. Ambience was good. The garlic bread was not that great. But the other dishes were yum.',
 'Rated 1.0, "  Quality of coffee has gone worse. So so so diluted coffee.. Though I ordered cold coffee, it was not at all cold. Even AC seemed to be off in this summer as we felt really hot through out our time. Instead of paying service taxes n getting horrible coffee n sweating, its better to buy coke or drink water or visit better cafes."',
 'Rated 3.0, "  Its a nice place to hangout because ambience is very relaxing. They serve different varieties of coffee. And recently theyve added sandwiches burgers . But theyre not UpTo the mark. The only thing I like here is toastizza!! Not worth the price."',
 'Rated 3.5,   Good place to hangout with buddies, friendly staff but service could be improved. I ordered Cool Blue beverage, sizzle dazzle brownie and burger were so yummy. I would definitely recommend this place.

In [463]:
#Reviews = Reviews.split("RATED\\n") #splitting at '), ' to split into individual tuple

In [543]:
Reviews[0]

'(Rated 4.0,   This outlet is better than other outlets. But the service was slow. Ambience was good. The garlic bread was not that great. But the other dishes were yum.'

In [544]:
type(Reviews)

list

In [685]:
#Reviews = Reviews.split("\'), ")

In [435]:
#Reviews

In [545]:
len(Reviews)

9

In [546]:
Reviews_df = pd.DataFrame({'Text':Reviews})

In [547]:
Reviews_df # each row has a tuple containing rating and a review text

,Text
0,"(Rated 4.0, This outlet is better than other..."
1,"Rated 1.0, "" Quality of coffee has gone worse..."
2,"Rated 3.0, "" Its a nice place to hangout beca..."
3,"Rated 3.5, Good place to hangout with buddie..."
4,"Rated 3.5, Last month we visited this place ..."
5,"Rated 3.0, "" This Cafe Coffee Day is a stones..."
6,"Rated 4.5, Like Shubha has stated, this coff..."
7,"Rated 4.0, "" I know this is like any other co..."
8,"Rated 3.5, Cafe! best place to hang out wid ..."


In [548]:
#now we need to split each tuple into 2 columns # after investigation we find that row 8 was actually part of 7
#but got in a new row due to splitting criteria

In [549]:
Reviews_df['Rating'] = Reviews_df.Text.apply(lambda x: x.split(', ')[0])

In [550]:
Reviews_df['Review'] = Reviews_df.Text.apply(lambda x: x.split(', ')[1])

In [552]:
Reviews_df

,Text,Rating,Review
0,"(Rated 4.0, This outlet is better than other...",(Rated 4.0,This outlet is better than other outlets. Bu...
1,"Rated 1.0, "" Quality of coffee has gone worse...",Rated 1.0,""" Quality of coffee has gone worse. So so so ..."
2,"Rated 3.0, "" Its a nice place to hangout beca...",Rated 3.0,""" Its a nice place to hangout because ambienc..."
3,"Rated 3.5, Good place to hangout with buddie...",Rated 3.5,Good place to hangout with buddies
4,"Rated 3.5, Last month we visited this place ...",Rated 3.5,Last month we visited this place
5,"Rated 3.0, "" This Cafe Coffee Day is a stones...",Rated 3.0,""" This Cafe Coffee Day is a stones throw from..."
6,"Rated 4.5, Like Shubha has stated, this coff...",Rated 4.5,Like Shubha has stated
7,"Rated 4.0, "" I know this is like any other co...",Rated 4.0,""" I know this is like any other coffee days b..."
8,"Rated 3.5, Cafe! best place to hang out wid ...",Rated 3.5,Cafe! best place to hang out wid best buddie...


In [473]:
#We need to clean both the rating and review columns

In [553]:
def clean_text(x):
    text = x.replace("Rated", "")
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,']", "", text)
    text = text.lower()
    return text

In [554]:
Reviews_df['Rating'] = Reviews_df['Rating'].apply(clean_text)

In [555]:
Reviews_df.Rating = Reviews_df.Rating.astype(float)

In [556]:
Reviews_df.Rating = round(Reviews_df.Rating/10,1)

In [557]:
def clean_text2(text):

    text = text.lower().strip()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"<br>", " ", text)
    text = re.sub(r"([-?.!,/\"])", r" \1 ", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,'*\\]", "", text)
    text = re.sub(r"[ ]+", " ", text)
    text = text.rstrip().strip()

    return text

In [558]:
Reviews_df.Review = Reviews_df.Review.apply(clean_text2)

In [559]:
corpus = " ".join(i for i in Reviews_df.Review)

In [560]:
corpus # when this corpus is lrge enough we can count the number of words occurring max times

'this outlet is better than other outlets but the service was slow ambience was good the garlic bread was not that great but the other dishes were yum quality of coffee has gone worse so so so diluted coffee though i ordered cold coffee its a nice place to hangout because ambience is very relaxing they serve different varieties of coffee and recently theyve added sandwiches burgers but theyre not upto the mark the only thing i like here is toastizza not worth the price good place to hangout with buddies last month we visited this place this cafe coffee day is a stones throw from my residence it is better than other outlets in some ways like shubha has stated i know this is like any other coffee days but since this is katriguppe and my area i hold it very close to menni would want to visit this place any day anytime alone or with anybody for that matter and it sure does gimme a good feelnnmy personal favourite hot cappuccino cafe best place to hang out wid best buddies'

In [561]:
#as we can see we only have a small number of reviews - 9 rows on which no modelling will be effective. 
#But what we can do is we can find the number of rows/len for each restaurant and which ever has a lot we can try and do
#sentiment analysis for that restaurant

#we can combine all ccd outlets to get sentiment but that will not be good since outlets are independent.

In [580]:
len(data.name)

51717

In [581]:
data.columns.values

array(['name', 'online_order', 'book_table', 'rate', 'votes', 'rest_type',
       'dish_liked', 'cuisines', 'approx_cost(for two people)', 'Reviews',
       'menu_item', 'Type', 'City'], dtype=object)

In [591]:
#we will just need the Reviews column for further study and hence creating a new dataframe with only that column
data_new = data[['name','Reviews']]

In [594]:
data_new.rename(columns= {'name': 'Name'}, inplace = True)

In [596]:
data_new.head(2)

,Name,Reviews
0,Jalsa,"[('Rated 4.0', 'RATED\n A beautiful place to ..."
1,Spice Elephant,"[('Rated 4.0', 'RATED\n Had been here for din..."


In [ ]:
CCD1.Reviews= CCD1.Reviews.replace("RATED\\n", "")
CCD1.Reviews = CCD1.Reviews.replace("\'", "")
Reviews = CCD1.Reviews.strip('[]')
Reviews = Reviews.split("), (")
Reviews_df = pd.DataFrame({'Text':Reviews})
Reviews_df['Rating'] = Reviews_df.Text.apply(lambda x: x.split(', ')[0])
Reviews_df['Review'] = Reviews_df.Text.apply(lambda x: x.split(', ')[1])
def clean_text(x):
    text = x.replace("Rated", "")
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,']", "", text)
    text = text.lower()
    return text
Reviews_df['Rating'] = Reviews_df['Rating'].apply(clean_text)
Reviews_df.Rating = Reviews_df.Rating.astype(float)
Reviews_df.Rating = round(Reviews_df.Rating/10,1)


In [ ]:
def clean_text2(text):

    text = text.lower().strip()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"<br>", " ", text)
    text = re.sub(r"([-?.!,/\"])", r" \1 ", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,'*\\]", "", text)
    text = re.sub(r"[ ]+", " ", text)
    text = text.rstrip().strip()

    return text
Reviews_df.Review = Reviews_df.Review.apply(clean_text2)

In [600]:
#def review_rating(name):
 #   df = data_new[data_new.Name == name]
  #  return df
    
#review_rating("Cafe Coffee Day")

In [673]:
data_new.Reviews[0]

'[(\'Rated 4.0\', \'RATED\\n  A beautiful place to dine in.The interiors take you back to the Mughal era. The lightings are just perfect.We went there on the occasion of Christmas and so they had only limited items available. But the taste and service was not compromised at all.The only complaint is that the breads could have been better.Would surely like to come here again.\'), (\'Rated 4.0\', \'RATED\\n  I was here for dinner with my family on a weekday. The restaurant was completely empty. Ambience is good with some good old hindi music. Seating arrangement are good too. We ordered masala papad, panner and baby corn starters, lemon and corrionder soup, butter roti, olive and chilli paratha. Food was fresh and good, service is good too. Good for family hangout.\\nCheers\'), (\'Rated 2.0\', \'RATED\\n  Its a restaurant near to Banashankari BDA. Me along with few of my office friends visited to have buffet but unfortunately they only provide veg buffet. On inquiring they said this plac

In [661]:
data_new.iloc[1]

Name                                          Spice Elephant
Reviews    [('Rated 4.0', 'RATED\n  Had been here for din...
Name: 1, dtype: object

In [786]:
#some restaurants don't have any review. it's missing. let's remove them. only then the below fucntion will work
data_new = data_new[data_new.Reviews != '[]']
data_new.shape

(44122, 2)

In [873]:
a = data_new.iloc[1688,]

In [861]:
a['Reviews']

'[(\'Rated 4.0\', \'RATED\\n  Its a really good restaurant but cost is on a higher factor. They are good south indian dish maker but have also delivered some very good dishes like Kashmiri Pulav. Self service model is ok and dine in facility is very good. Nice ambience for dine in.\'), (\'Rated 2.0\', \'RATED\\n  Had ordered plain Dosa and coffee. The plain Dosa along with the sambhar and chutneys were salty and oily. The order also took a long time. The coffee had less sugar and was not good. The prices are on the higher side and it seems we are paying for the ambience and not for quality of food.\\n\\nThe mini meal was good. Quantity was sufficient for 1. It had chappathi, 3 rice OEMs, sweet and fried item like papad\'), (\'Rated 5.0\', "RATED\\n  Love this place for it\'s South Indian preparations , it\'s our Sunday morning breakfast destination. Softest idlies , cripy dosas, yummy chutneys , spicy sambar ,also mini tiffin is a complete delight with kesar bath and pongal made in ghe

In [874]:
a
Reviews = a.Reviews.replace("RATED\\n", "")
Reviews = Reviews.replace("\'", "")
Reviews = Reviews.strip('[]')
Reviews = Reviews.split("), (R")
Reviews_df = pd.DataFrame({'Text':Reviews})
Reviews_df['Rating'] = Reviews_df.Text.apply(lambda x: x.split(', ')[0])
Reviews_df['Review'] = Reviews_df.Text.apply(lambda x: x.split(', ')[1])

In [876]:
def clean_text(x):
    text = x.replace("ated", "")
    text = text.replace("R", "")
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,']", "", text)
    text = text.lower()
    return text

In [877]:
Reviews_df['Rating'] = Reviews_df['Rating'].apply(clean_text)

In [879]:
Reviews_df.Rating = Reviews_df.Rating.astype(float)
Reviews_df.Rating = round(Reviews_df.Rating/10,1)

In [880]:
Reviews_df

,Text,Rating,Review
0,"(Rated 4.0, Its a really good restaurant but...",4.0,Its a really good restaurant but cost is on ...
1,"ated 2.0, Had ordered plain Dosa and coffee....",2.0,Had ordered plain Dosa and coffee. The plain...
2,"ated 5.0, "" Love this place for its South Ind...",5.0,""" Love this place for its South Indian prepar..."
3,"ated 3.0, "" Ambiance - 3.5/5\nTaste - 3.5/5\n...",3.0,""" Ambiance - 3.5/5\nTaste - 3.5/5\nStaff - 4...."
4,"ated 5.0, We always have delicious and pipin...",5.0,We always have delicious and piping hot dish...
5,"ated 3.0, Have been regular to this place .G...",3.0,Have been regular to this place .Generally f...
6,"ated 3.0, This is again a typical A2b experi...",3.0,This is again a typical A2b experience in te...
7,"ated 4.0, Ordered South Indian Thali and Nor...",4.0,Ordered South Indian Thali and North Indian ...


In [883]:
def custom_function(value):
    df = data_new.iloc[value]
    name = df.Name
    Reviews = df.Reviews.replace("RATED\\n", "")
    Reviews = Reviews.replace("\'", "")
    Reviews = Reviews.strip('[]')
    Reviews = Reviews.split("), (R")
    Reviews_df = pd.DataFrame({'Text':Reviews})
    Reviews_df['Rating'] = Reviews_df.Text.apply(lambda x: x.split(', ')[0])
    Reviews_df['Review'] = Reviews_df.Text.apply(lambda x: x.split(', ')[1])
    
    def clean_text(x):
        text = x.replace("ated", "")
        text = text.replace("R", "")
        text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,']", "", text)
        text = text.lower()
        return text
    Reviews_df['Rating'] = Reviews_df['Rating'].apply(clean_text)
    Reviews_df.Rating = Reviews_df.Rating.astype(float)
    Reviews_df.Rating = round(Reviews_df.Rating/10,1)
        
    def clean_text2(text):
        text = text.lower().strip()
        text = re.sub(r"i'm", "i am", text)
        text = re.sub(r"he's", "he is", text)
        text = re.sub(r"she's", "she is", text)
        text = re.sub(r"that's", "that is", text)
        text = re.sub(r"what's", "what is", text)
        text = re.sub(r"where's", "where is", text)
        text = re.sub(r"how's", "how is", text)
        text = re.sub(r"\'s", " is", text)
        text = re.sub(r"\'ll", " will", text)
        text = re.sub(r"\'ve", " have", text)
        text = re.sub(r"\'re", " are", text)
        text = re.sub(r"\'d", " would", text)
        text = re.sub(r"won't", "will not", text)
        text = re.sub(r"can't", "cannot", text)
        text = re.sub(r"n't", " not", text)
        #text = re.sub(r"Rated)
        text = re.sub(r"<br>", " ", text)
        text = re.sub(r"([-?.!,/\"])", r" \1 ", text)
        text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,'*\\]", "", text)
        text = re.sub(r"[ ]+", " ", text)
        text = text.rstrip().strip()
        return text
      
    Reviews_df.Review = Reviews_df.Review.apply(clean_text2)
    #print("Name of the restaurant is :", name)
    #print("Number of rows in the Dataframe :", Reviews_df.shape[0])
    #return len(Reviews_df)
    return Reviews_df.shape[0]

In [884]:
custom_function(1688) #put any value between 0 and len(data) and the corresponding restaurant with its reviews and 
                    #customer ratings will appear

8

In [734]:
len(data_new)

51717

In [887]:
x = [custom_function(i) for i in range(0, len(data_new))]

In [899]:
values_df = pd.DataFrame({'Values':x})

In [911]:
values_df.tail(10) # with 1099 reviews we can still go on and use textblob to classify them as pos neg and neutral
#and then train using Multinomial Naive Bayes

,Values
44112,1099
44113,1102
44114,1181
44115,1214
44116,1265
44117,1285
44118,1485
44119,1515
44120,1742
44121,1769
